### Crawing Daily Trading Information of Institutions

In [6]:
import requests
from bs4 import BeautifulSoup
res = requests.get('http://www.twse.com.tw/fund/BFI82U?response=json&dayDate=&weekDate=&monthDate=&type=day&_=1522465284842')
if res.encoding != 'utf-8':
    res.encoding = 'utf-8'

print(res.text)

{"stat":"OK","title":"107年04月03日 三大法人買賣金額統計表","fields":["單位名稱","買進金額","賣出金額","買賣差額"],"date":"20180403","data":[["自營商(自行買賣)","1,332,075,890","1,527,085,313","-195,009,423"],["自營商(避險)","5,053,878,133","5,848,013,670","-794,135,537"],["投信","877,499,250","1,641,700,150","-764,200,900"],["外資及陸資(不含外資自營商)","26,111,675,637","35,014,554,368","-8,902,878,731"],["外資自營商","236,373,220","52,695,290","183,677,930"],["合計","33,375,128,910","44,031,353,501","-10,656,224,591"]],"params":{"response":"json","dayDate":"20180403","weekDate":"20180402","monthDate":"20180403","type":"day","_":"1522465284842","controller":"fund","format":null,"action":"BFI82U","lang":"zh"},"notes":["自營商表示證券自營商專戶。","投信表示本國投資信託基金。","外資及陸資表示依「華僑及外國人投資證券管理辦法」及「大陸地區投資人來臺從事證券投資及期貨交易管理辦法」辦理登記等投資人。","因外資自營商買賣金額已計入自營商買賣金額，故不納入三大法人買賣金額之合計數計算。","本統計資訊含一般、零股、盤後定價、鉅額，不含拍賣、標購。","本資訊以當日原始成交情形統計，不以證券商申報錯帳、更正帳號等調整後資料統計。","外幣成交值係以本公司當日下午3時30分公告匯率換算後加入成交金額。<br>公告匯率請參考本公司首頁>交易資訊>雙幣ETF專區>代號對應及每日公告匯率。"]}


In [26]:
import json
dictionary = res.text.replace("'", "\"")
data = json.loads(dictionary)
print(data.keys())

dict_keys(['stat', 'title', 'fields', 'date', 'data', 'params', 'notes'])


In [27]:
import pandas as pd, numpy as np
pd.DataFrame(data['data'], columns=data['fields'])

,單位名稱,買進金額,賣出金額,買賣差額
0,自營商(自行買賣),"1,332,075,890","1,527,085,313","-195,009,423"
1,自營商(避險),"5,053,878,133","5,848,013,670","-794,135,537"
2,投信,"877,499,250","1,641,700,150","-764,200,900"
3,外資及陸資(不含外資自營商),"26,111,675,637","35,014,554,368","-8,902,878,731"
4,外資自營商,"236,373,220","52,695,290","183,677,930"
5,合計,"33,375,128,910","44,031,353,501","-10,656,224,591"


### 如何產生不同天期的交易資訊? Get Trading Info of Different Days

In [1]:
# http://www.largitdata.com/course/22/

In [2]:
from datetime import date, timedelta, datetime
import time 

print(date.today())
print(date.today() + timedelta(days=1))
print(date.today() + timedelta(days=-1))

2018-04-05
2018-04-06
2018-04-04


In [3]:
dayarray = str(date.today()).split('-') #根據 '-' 去分割
print(dayarray)
print('-'.join([str(int(dayarray[0])-1911), dayarray[1], dayarray[2]])) #在每個element 中間以 '-'連接

['2018', '04', '05']
107-04-05


In [4]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np

url = 'http://www.twse.com.tw/fund/BFI82U?response=html&dayDate={0}&weekDate={0}&monthDate={0}&type=day'

def getTradeValue(dt):
#     res = requests.get(url.format(dt))
#     if res.encoding != 'utf-8':
#         res.encoding = 'utf-8'
#     dictionary = res.text.replace("'", "\"")
#     print(dictionary)
#     data = json.loads(dictionary)
#     data = pd.DataFrame(data['data'], columns=data['fields'])
    data = pd.read_html(url.format(dt))[0]
    data.columns = data.columns.droplevel()
    return data
    
getTradeValue(str(date.today() - timedelta(days=3)).replace('-', ''))

,單位名稱,買進金額,賣出金額,買賣差額
0,自營商(自行買賣),1335596870,1352466850,-16869980
1,自營商(避險),4847110815,5949424569,-1102313754
2,投信,1119585900,1393401480,-273815580
3,外資及陸資(不含外資自營商),16729913873,18839357363,-2109443490
4,外資自營商,140539490,245214820,-104675330
5,合計,24032207458,27534650262,-3502442804


In [6]:
today = date.today()

for i in range(1, 3):
    today = today + timedelta(days=-1)
    dayarray = str(today).replace('-', '')
    try: 
        print(dayarray)
        print(getTradeValue(dayarray))
        print('\n', '-'*20)
    except:
        continue

20180404
20180403
             單位名稱         買進金額         賣出金額         買賣差額
0       自營商(自行買賣)   1332075890   1527085313   -195009423
1         自營商(避險)   5053878133   5848013670   -794135537
2              投信    877499250   1641700150   -764200900
3  外資及陸資(不含外資自營商)  26111675637  35014554368  -8902878731
4           外資自營商    236373220     52695290    183677930
5              合計  33375128910  44031353501 -10656224591

 --------------------


### Store All Data into MobgoDB

In [1]:
'''
id: int
item: str
total_buy: int
total_sell: int
difference: int
date: datetime
'''
from pymongo import MongoClient
import pymongo
from datetime import date, timedelta, datetime
import json
import pandas as pd
import numpy as np
from bson.objectid import ObjectId
import pprint
from tqdm import tqdm
import time

In [2]:
client = MongoClient('mongodb://finance:27017/')
db = client.stock
collect = db['institutional-investors-trading-values']
# collect.create_index('date', unique=True)

In [5]:
url = 'http://www.twse.com.tw/fund/BFI82U?response=html&dayDate={0}&weekDate={0}&monthDate={0}&type=day'

def getTradeValue(dt):
    data = pd.read_html(url.format(dt))[0]
    data.columns = data.columns.droplevel()
    return data

def str_to_json(x):
    dictionary = x.replace("'", "\"")
    data = json.loads(dictionary)
    return data
    
day = str(date.today() - timedelta(days=10))
data = getTradeValue(day.replace('-', ''))
data['date'] = day
data.columns = ['institution', 'total_buy', 'total_sell', 'net', 'date']
data.apply(lambda s: str_to_json(s.to_json()), axis=1).values.tolist()

[{'date': '2018-03-30',
  'institution': '自營商(自行買賣)',
  'net': -688266239,
  'total_buy': 1363463280,
  'total_sell': 2051729519},
 {'date': '2018-03-30',
  'institution': '自營商(避險)',
  'net': 965034063,
  'total_buy': 5886687328,
  'total_sell': 4921653265},
 {'date': '2018-03-30',
  'institution': '投信',
  'net': -451886027,
  'total_buy': 1432709696,
  'total_sell': 1884595723},
 {'date': '2018-03-30',
  'institution': '外資及陸資(不含外資自營商)',
  'net': 1956930179,
  'total_buy': 21635169030,
  'total_sell': 19678238851},
 {'date': '2018-03-30',
  'institution': '外資自營商',
  'net': -34348660,
  'total_buy': 233487630,
  'total_sell': 267836290},
 {'date': '2018-03-30',
  'institution': '合計',
  'net': 1781811976,
  'total_buy': 30318029334,
  'total_sell': 28536217358}]

In [6]:
requestDate = datetime(2018, 3, 25)
crawled = []

# reset collection
collect.delete_many({})

runs = (datetime.today() - requestDate).days
pbar = tqdm(total = runs+1)
while requestDate < datetime.today():
    requestDate = requestDate + timedelta(days=1)
    day = datetime.strftime(requestDate, '%Y-%m-%d')
    print(requestDate)
    
    time.sleep(5)
    try: 
        data = getTradeValue(day.replace('-', ''))
        data['date'] = day
        data.columns = ['institution', 'total_buy', 'total_sell', 'net', 'date']
        crawled.extend(data.apply(lambda s: str_to_json(s.to_json()), axis=1).values.tolist())
        pbar.update(1)
    except:
        continue

pbar.close()
collect.insert_many(crawled)

  0%|          | 0/16 [00:00<?, ?it/s]

2018-03-26 00:00:00


  6%|▋         | 1/16 [00:05<01:16,  5.08s/it]

2018-03-27 00:00:00


 12%|█▎        | 2/16 [00:10<01:10,  5.07s/it]

...

2018-04-03 00:00:00


 50%|█████     | 8/16 [00:45<00:49,  6.13s/it]

2018-04-04 00:00:00
2018-04-05 00:00:00
2018-04-06 00:00:00
2018-04-07 00:00:00
2018-04-08 00:00:00
2018-04-09 00:00:00


 56%|█████▋    | 9/16 [01:16<01:33, 13.42s/it]

2018-04-10 00:00:00


In [3]:
i = 0
for j in collect.find():
    pprint.pprint(j)
    i += 1
print(i)

{'_id': ObjectId('5acb11bce88ed50c6346c030'),
 'date': '2018-04-02',
 'institution': '自營商(自行買賣)',
 'net': -16869980,
 'total_buy': 1335596870,
 'total_sell': 1352466850}
{'_id': ObjectId('5acb11bce88ed50c6346c031'),
 'date': '2018-04-02',
 'institution': '自營商(避險)',
 'net': -1102313754,
 'total_buy': 4847110815,
 'total_sell': 5949424569}
{'_id': ObjectId('5acb11bce88ed50c6346c032'),
 'date': '2018-04-02',
 'institution': '投信',
 'net': -273815580,
 'total_buy': 1119585900,
 'total_sell': 1393401480}
{'_id': ObjectId('5acb11bce88ed50c6346c033'),
 'date': '2018-04-02',
 'institution': '外資及陸資(不含外資自營商)',
 'net': -2109443490,
 'total_buy': 16729913873,
 'total_sell': 18839357363}
{'_id': ObjectId('5acb11bce88ed50c6346c034'),
 'date': '2018-04-02',
 'institution': '外資自營商',
 'net': -104675330,
 'total_buy': 140539490,
 'total_sell': 245214820}
{'_id': ObjectId('5acb11bce88ed50c6346c035'),
 'date': '2018-04-02',
 'institution': '合計',
 'net': -3502442804,
 'total_buy': 24032207458,
 'total_sell'